# Lab Problems
**Simple Problem**
1. MNIST Dataset - location: https://huggingface.co/datasets/ylecun/mnist
2. Use weights and biases to view training. https://docs.wandb.ai/guides/integrations/lightning
3. CIFAR10 Dataset submission

**Intermediate Problem**
2. TinyImagenet - location: https://huggingface.co/datasets/zh-plus/tiny-imagenet

In [ ]:
!pip install lightning torchmetrics datasets wandb

# 1. DATASET DOWNLOADING

In [2]:
# Dataset
import torch, torch.nn as nn
import torchvision
import datasets as huggingface_datasets

def get_dataloaders():
    # Dataset is being downloaded from huggingface_datasets library
    training_dataset    = huggingface_datasets.load_dataset("mnist", split="train")
    validation_dataset  = huggingface_datasets.load_dataset("mnist", split="test" )

    class Custom_Dataset(torch.utils.data.Dataset):
        def __init__(self, dataset):
            self.dataset = dataset
            self.data_transformations = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                ])

        def __getitem__(self, index):
            image , label  = self.dataset[index]['image'], self.dataset[index]['label']
            image_tensor   = self.data_transformations(image)
            single_example = (image_tensor, label)
            return single_example

        def __len__(self):
            return len(self.dataset)

    custom_training_dataset   = Custom_Dataset(training_dataset) # __init__ method called automatically to initalize
    training_dataloader       = torch.utils.data.DataLoader(custom_training_dataset, batch_size = 3, shuffle= True)

    custom_validation_dataset = Custom_Dataset(validation_dataset)
    validation_dataloader     = torch.utils.data.DataLoader(custom_validation_dataset, batch_size = 32, shuffle= False)

    return training_dataloader, validation_dataloader

# TODO: Model ARCHITECTURE

In [3]:
import torch, torch.nn as nn
model = torch.nn.Sequential(
    # Convolution Block: Image is 2D
    nn.Conv2d(1 , 10, (3,3)), nn.ReLU(),  # Layer 1:
    nn.Conv2d(10, 20, (3,3)), nn.ReLU(),  # Layer 2:
    nn.Conv2d(20, 30, (3,3)), nn.ReLU(),  # Layer 3:
    nn.Conv2d(30, 40, (3,3)), nn.ReLU(),  # Layer 4:

    # Image is flattened to send to Linear Layers
    nn.Flatten(start_dim=1),

    # Decision Maker Block:
    # Layer 5:
    nn.Linear(16000, 10),
)

test_image = torch.randn((1,1,28,28))
model(test_image).shape

torch.Size([1, 10])

# MODEL TRAINING CODE

In [4]:
import lightning, torchmetrics

def get_lightning_model(input_pytorch_model):
    class Lightning_Module(lightning.LightningModule):
        def __init__(self, model):
            super().__init__()

            self.model = model
            self.automatic_optimization = False
            self.training_accuracy      = torchmetrics.Accuracy(task="multiclass", num_classes=10)
            self.validation_accuracy    = torchmetrics.Accuracy(task="multiclass", num_classes=10)


        def training_step (self, batch, batch_idx):
            images_actual, labels_actual        = batch
            predicted_logits  = self.model(images_actual)
            labels_predicted  = torch.argmax(predicted_logits, dim = 1)

            loss             = torch.nn.functional.cross_entropy(predicted_logits, labels_actual)

            optimizer        = self.optimizers()
            optimizer.zero_grad()
            self.manual_backward(loss)
            # Manually calculating Error's relationship with Model Parameters
            # dE_dW = torch.autograd.grad(outputs = loss, inputs = self.model.parameters())
            optimizer.step()
            # Manually updating parameters instead of optimizer updating them
            """
            for individual_parameter in self.parameters():
                individual_parameter = individual_parameter - individual_parameter.grad * learning_rate
            """
            self.training_accuracy(labels_predicted, labels_actual)
            self.log("train_loss"     , loss                   , prog_bar = True)
            self.log("train_accuracy" , self.training_accuracy , prog_bar = True)

            return loss

        def validation_step (self, batch, batch_idx):
            images_actual, labels_actual        = batch
            predicted_logits  = self.model(images_actual)
            labels_predicted  = torch.argmax(predicted_logits, dim = 1)

            loss = torch.nn.functional.cross_entropy(predicted_logits, labels_actual)
            self.validation_accuracy(labels_predicted, labels_actual)
            self.log("validation_loss"     , loss                     , prog_bar= False)
            self.log("validation_accuracy" , self.validation_accuracy , prog_bar= True)

        def configure_optimizers(self):
            optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
            return optimizer

    lightning_model = Lightning_Module(input_pytorch_model)
    return lightning_model


# TRAINING PROCESS

In [5]:
# Initialize wandb logger
from lightning.pytorch.loggers import WandbLogger
wandb_logger = WandbLogger(project='wandb-lightning', job_type='train')

In [6]:
training_dataloader, validation_dataloader = get_dataloaders()
lightning_model                            = get_lightning_model(model)

epochs  = 10
lightning_trainer = lightning.Trainer( max_epochs= epochs , logger=wandb_logger)

lightning_trainer.fit(model=lightning_model, train_dataloaders= training_dataloader, val_dataloaders= validation_dataloader);

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ajinkyakolhe112 (m2_mac). Use `wandb login --relogin` to force relogin



  | Name                | Type               | Params | Mode 
-------------------------------------------------------------------
0 | model               | Sequential         | 178 K  | train
1 | training_accuracy   | MulticlassAccuracy | 0      | train
2 | validation_accuracy | MulticlassAccuracy | 0      | train
-------------------------------------------------------------------
178 K     Trainable params
0         Non-trainable params
178 K     Total params
0.713     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]